In [1]:
!pip install unidecode tensorboardX
!git clone --recursive https://github.com/NVIDIA/flowtron

     |████████████████████████████████| 245kB 12.2MB/s 
     |████████████████████████████████| 317kB 20.4MB/s 
Cloning into 'flowtron'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 171 (delta 14), reused 16 (delta 5), pack-reused 135
Receiving objects: 100% (171/171), 2.81 MiB | 5.50 MiB/s, done.
Resolving deltas: 100% (64/64), done.
Submodule 'apex' (https://github.com/NVIDIA/apex) registered for path 'apex'
Submodule 'tacotron2' (https://github.com/NVIDIA/tacotron2) registered for path 'tacotron2'
Cloning into '/content/flowtron/apex'...
remote: Enumerating objects: 30, done.        
remote: Counting objects: 100% (30/30), done.        
remote: Compressing objects: 100% (30/30), done.        
remote: Total 7865 (delta 7), reused 10 (delta 0), pack-reused 7835        
Receiving objects: 100% (7865/7865), 13.99 MiB | 13.90 MiB/s, done.
Resolving deltas: 100% (5356/5356), done.
Clon

In [2]:
%cd flowtron

/content/flowtron


In [3]:
!mkdir models

In [4]:
import sys
import json
import torch
import gdown
import matplotlib

from data import Data
import IPython.display as ipd
from flowtron import Flowtron
import matplotlib.pylab as plt
from train import update_params
from torch.distributions import Normal

sys.path.insert(0, "tacotron2")
sys.path.insert(0, "tacotron2/waveglow")

from denoiser import Denoiser

In [5]:
gdown.download('https://drive.google.com/uc?id=1Cjd6dK_eFz6DE0PKXKgKxrzTUqzzUDW-', 'models/flowtron_ljs.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF', 'models/waveglow_256channels_universal_v5.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=100YJu80Y-k5katrwzzE6rFoEHJ2rLmkc', 'data/surprised_samples.zip', quiet=False)
%cd data
!unzip surprised_samples.zip
%cd ..

Downloading...
From: https://drive.google.com/uc?id=1Cjd6dK_eFz6DE0PKXKgKxrzTUqzzUDW-
To: /content/flowtron/models/flowtron_ljs.pt
244MB [00:02, 82.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
To: /content/flowtron/models/waveglow_256channels_universal_v5.pt
676MB [00:09, 72.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=100YJu80Y-k5katrwzzE6rFoEHJ2rLmkc
To: /content/flowtron/data/surprised_samples.zip
100%|██████████| 539k/539k [00:00<00:00, 79.0MB/s]

/content/flowtron/data
Archive:  surprised_samples.zip
   creating: surprised_samples/
  inflating: surprised_samples/LICENSE  
   creating: __MACOSX/
   creating: __MACOSX/surprised_samples/
  inflating: __MACOSX/surprised_samples/._LICENSE  
  inflating: surprised_samples/03-01-08-02-01-01-24.wav  
  inflating: surprised_samples/03-01-08-01-02-01-24.wav  
  inflating: surprised_samples/03-01-08-02-02-02-24.wav  
  inflating: surprised_samples/03-01-08-01-01-02-24.wav  
  inflating: surprised_samples/03-01-08-01-02-02-24.wav  
  inflating: surprised_samples/03-01-08-02-01-02-24.wav  
  inflating: surprised_samples/03-01-08-01-01-01-24.wav  
  inflating: surprised_samples/03-01-08-02-02-01-24.wav  
  inflating: surprised_samples/surprised_audiofilelist_text.txt  
  inflating: __MACOSX/surprised_samples/._surprised_audiofilelist_text.txt  
/content/flowtron


### Load FlowTron

In [6]:
config_path = "config.json"
params = ["model_config.dummy_speaker_embedding=0",
          "data_config.p_arpabet=1.0"]

with open(config_path) as f:
    data = f.read()

config = json.loads(data)
update_params(config, params)

data_config = config["data_config"]
model_config = config["model_config"]

model_config.dummy_speaker_embedding=0
dummy_speaker_embedding=0
data_config.p_arpabet=1.0
p_arpabet=1.0


In [7]:
model_path = "models/flowtron_ljs.pt"
state_dict = torch.load(model_path, map_location='cpu')['state_dict']
model = Flowtron(**model_config)
model.load_state_dict(state_dict)
_ = model.eval().cuda()

### Load WaveGlow

In [8]:
waveglow_path = 'models/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
_ = waveglow.eval().cuda()
denoiser = Denoiser(waveglow).cuda().eval()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

### Prep DataLoader

In [9]:
dataset_path = 'data/surprised_samples/surprised_audiofilelist_text.txt'
dataset = Data(
    dataset_path,
    **dict((k, v) for k, v in data_config.items() if k not in ['training_files', 'validation_files']))

Number of speakers : 1
Number of speakers : 1
